import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

In [2]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=42
btch=80
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=NonInvasiveFetalECGThorax1(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
print('shape ecg=',np.shape(ecg))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (1800, 751)
first_test_shape= (1965, 751)
classes_quantity= 42
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42}
Count_labels= [36 52 45 40 48 44 36 50 46 56 36 39 46 42 40 39 52 45 40 49 43 40 35 41
 41 37 47 40 47 40 43 41 37 52 45 40 37 41 43 45 39 45]
max(train_feature_Altitude)= 42.0
min(train_feature_Altitude)= -5.7322181
first_train_sample=
 [ 3.3000000e+01  3.0577079e+00  2.9885631e+00  2.8214720e+00
  2.6392369e+00  2.4456015e+00  2.1598024e+00  1.7604417e+00
  1.2885524e+00  7.7969192e-01  2.5702086e-01 -2.4136675e-01
 -6.9723052e-01 -1.1682231e+00 -1.7182241e+00 -2.3618027e+00
 -3.0698550e+00 -3.7349237e+00 -4.2844303e+00 -4.7071089e+00
 -5.0354394e+00 -5.2173834e+00 -5.2658047e+00 -5.2337997e+00
 -5.1379266e+00 -5.0083663e+00 -4.8910492e+00 -4.7745101e+00
 -4.5631708e+00 -4.2865916e+00 -4.0250031e+00 -3.7163384e+00
 -3.3250977e+00 -2.957

1 36	2 52	3 45	4 40	5 48	6 44	7 36	8 50	9 46	10 56	11 36	12 39	13 46	14 42	15 40	16 39	17 52	18 45	19 40	20 49	21 43	22 40	23 35	24 41	25 41	26 37	27 47	28 40	29 47	30 40	31 43	32 41	33 37	34 52	35 45	36 40	37 37	38 41	39 43	40 45	41 39	42 45	shape ecg= (1800, 481)


<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [4]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.empty((0,smpl_rte+1))

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    windws=np.append(windws,vars()['wndws'+str(cls)],axis=0)
    
print("each class and its windows =")

furir_windws0=np.fft.fft(np.array(windws[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([windws[:,-1]])),axis=1)
print("shape furir_windws=", np.shape(furir_windws))

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
shape furir_windws0= (1800, 480)
shape furir_windws= (1800, 481)
1 36	2 52	3 45	4 40	5 48	6 44	7 36	8 50	9 46	10 56	11 36	12 39	13 46	14 42	15 40	16 39	17 52	18 45	19 40	20 49	21 43	22 40	23 35	24 41	25 41	26 37	27 47	28 40	29 47	30 40	31 43	32 41	33 37	34 52	35 45	36 40	37 37	38 41	39 43	40 45	41 39	42 45	
 max = 56


In [5]:
#clses=3                                   #تعیین میزان افزایش نمونه (تولید داده)
mx_wndws=int(mx_wndws+1)  #mx_wndws*0.3  # =classes-1 برای بررسی الگوریتم تعداد کمی کلاس آزمایش شد
print(mx_wndws)

57


<h1><center><div style="direction:rtl;font-family:B Nazanin">Augment Random CNN2</div></center></h1>

In [10]:
rpt_cnn_clsfctn=5
rslt=np.array([])
rslts=np.array([])
xtrain=np.empty((0,len(wndws1[0])), float) 


for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug10'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug11'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug12'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug20'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug21'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug22'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug30'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug31'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug32'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug40'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug41'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug42'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug50'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug51'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug52'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    #print("vlum_win=",vlum_win)
    vars()['ecg_aug10'+str(i)]=np.append(vars()['ecg_aug10'+str(i)],scl_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug11'+str(i)]=np.append(vars()['ecg_aug11'+str(i)],scl_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug12'+str(i)]=np.append(vars()['ecg_aug12'+str(i)],scl_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug20'+str(i)]=np.append(vars()['ecg_aug20'+str(i)],MgWrp_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug21'+str(i)]=np.append(vars()['ecg_aug21'+str(i)],MgWrp_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug22'+str(i)]=np.append(vars()['ecg_aug22'+str(i)],MgWrp_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug30'+str(i)]=np.append(vars()['ecg_aug30'+str(i)],TimWrp_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug31'+str(i)]=np.append(vars()['ecg_aug31'+str(i)],TimWrp_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug32'+str(i)]=np.append(vars()['ecg_aug32'+str(i)],TimWrp_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug40'+str(i)]=np.append(vars()['ecg_aug40'+str(i)],GDO_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug41'+str(i)]=np.append(vars()['ecg_aug41'+str(i)],GDO_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug42'+str(i)]=np.append(vars()['ecg_aug42'+str(i)],GDO_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug50'+str(i)]=np.append(vars()['ecg_aug50'+str(i)],frqnc_rndm_beat(furir_windws,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug51'+str(i)]=np.append(vars()['ecg_aug51'+str(i)],frqnc_rndm_beat(furir_windws,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug52'+str(i)]=np.append(vars()['ecg_aug52'+str(i)],frqnc_rndm_beat(furir_windws,i,vlum_win,mag=.2),axis=0)    

for methd in range(1,6):
    for Mgntud in range(0,3):
        xtrain=np.empty((0,len(wndws1[0])), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (1,cls_num+1):   
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,vars()['ecg_aug'+str(methd)+str(Mgntud)+str(i)],axis=0)
                #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            cls_wndws=np.array([])
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                cls_wndws=np.append(cls_wndws,wns)
                #print(cls, wns, end='\t')
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
            #print('\n max instance in classes of test windows =', mx_wndws)

            #print('min tst =', np.min(wndws_test[:,:-1]))
            #print('max tst =', np.max(wndws_test[:,:-1]))

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            #fullprint(ytest)

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(Xtrain)
            y_train=np.array(ytrain)
            X_test=np.array(Xtest)
            y_test=np.array(ytest)
            X_valid=np.array(Xtrain)
            y_valid=np.array(ytrain)
            #print('X_train => ', X_train.shape)
            #print('y_train => ', y_train.shape)
            #print('X_test  => ', X_test.shape)
            #print('y_test  => ', y_test.shape)
            #print('X_valid  => ', X_valid.shape)
            #print('y_valid  => ', y_valid.shape)

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            #Yildirin

            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data



            model_crs = Sequential()
            #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=3))

            model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
            #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


            model_crs.add(Flatten())
            model_crs.add(Dropout(0.1))
            model_crs.add(Dense(units=512,activation="relu"))
            #model_crs.add(Dense(units=2048,activation="relu"))
            model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)


            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(methd)+'  Mgntud '+str(Mgntud)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

C:\Users\user\Thesis\Classifying\CNNsplit\CNN_ECGNonInvasivefetal\augment.py:658: ComplexWarning: Casting complex values to real discards the imaginary part
  p_wn = np.float16(np.concatenate((P,pl), axis=1))


62/62 [==============================] - 0s 2ms/step


In [11]:
rslts

array(['mthd 1  Mgntud 0  average= [88 87 87]  max= [90 89 90]  var= 6',
       'mthd 1  Mgntud 1  average= [87 87 87]  max= [92 92 92]  var= 8',
       'mthd 1  Mgntud 2  average= [86 86 86]  max= [90 90 90]  var= 8',
       'mthd 2  Mgntud 0  average= [88 88 88]  max= [90 90 90]  var= 1',
       'mthd 2  Mgntud 1  average= [87 86 87]  max= [91 90 91]  var= 8',
       'mthd 2  Mgntud 2  average= [87 87 87]  max= [90 90 90]  var= 3',
       'mthd 3  Mgntud 0  average= [87 86 86]  max= [91 91 91]  var= 10',
       'mthd 3  Mgntud 1  average= [86 86 86]  max= [91 91 91]  var= 20',
       'mthd 3  Mgntud 2  average= [88 87 87]  max= [90 90 90]  var= 5',
       'mthd 4  Mgntud 0  average= [87 87 87]  max= [89 89 89]  var= 7',
       'mthd 4  Mgntud 1  average= [87 87 87]  max= [89 89 89]  var= 2',
       'mthd 4  Mgntud 2  average= [85 84 85]  max= [91 90 91]  var= 21',
       'mthd 5  Mgntud 0  average= [88 87 87]  max= [91 91 91]  var= 5',
       'mthd 5  Mgntud 1  average= [87 87 87]  m

<h1><center><div style="direction:rtl;font-family:B Nazanin">Augment Marginal CNN2</div></center></h1>

In [12]:
rpt_cnn_clsfctn=5
rslt=np.array([])
rslts=np.array([])
xtrain=np.empty((0,len(wndws1[0])), float) 
rprt=np.empty((0,3),float)


for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug10'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug11'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug12'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug20'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug21'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug22'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug30'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug31'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug32'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug40'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug41'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug42'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug50'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug51'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug52'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    #print("vlum_win=",vlum_win)
    vars()['ecg_aug10'+str(i)]=np.append(vars()['ecg_aug10'+str(i)],sclRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug11'+str(i)]=np.append(vars()['ecg_aug11'+str(i)],sclRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug12'+str(i)]=np.append(vars()['ecg_aug12'+str(i)],sclRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug20'+str(i)]=np.append(vars()['ecg_aug20'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug21'+str(i)]=np.append(vars()['ecg_aug21'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug22'+str(i)]=np.append(vars()['ecg_aug22'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug30'+str(i)]=np.append(vars()['ecg_aug30'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug31'+str(i)]=np.append(vars()['ecg_aug31'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug32'+str(i)]=np.append(vars()['ecg_aug32'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug40'+str(i)]=np.append(vars()['ecg_aug40'+str(i)],GDORaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug41'+str(i)]=np.append(vars()['ecg_aug41'+str(i)],GDORaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug42'+str(i)]=np.append(vars()['ecg_aug42'+str(i)],GDORaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug50'+str(i)]=np.append(vars()['ecg_aug50'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug51'+str(i)]=np.append(vars()['ecg_aug51'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug52'+str(i)]=np.append(vars()['ecg_aug52'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.2),axis=0)    

for methd in range(1,6):
    for Mgntud in range(0,3):
        xtrain=np.empty((0,len(wndws1[0])), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (1,cls_num+1):   
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,vars()['ecg_aug'+str(methd)+str(Mgntud)+str(i)],axis=0)
                #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            cls_wndws=np.array([])
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                cls_wndws=np.append(cls_wndws,wns)
                #print(cls, wns, end='\t')
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
            #print('\n max instance in classes of test windows =', mx_wndws)

            #print('min tst =', np.min(wndws_test[:,:-1]))
            #print('max tst =', np.max(wndws_test[:,:-1]))

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            #fullprint(ytest)

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(Xtrain)
            y_train=np.array(ytrain)
            X_test=np.array(Xtest)
            y_test=np.array(ytest)
            X_valid=np.array(Xtrain)
            y_valid=np.array(ytrain)
            #print('X_train => ', X_train.shape)
            #print('y_train => ', y_train.shape)
            #print('X_test  => ', X_test.shape)
            #print('y_test  => ', y_test.shape)
            #print('X_valid  => ', X_valid.shape)
            #print('y_valid  => ', y_valid.shape)

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            #Yildirin

            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data



            model_crs = Sequential()
            #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=3))

            model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
            #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


            model_crs.add(Flatten())
            model_crs.add(Dropout(0.1))
            model_crs.add(Dense(units=512,activation="relu"))
            #model_crs.add(Dense(units=2048,activation="relu"))
            model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)


            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(methd)+'  Mgntud '+str(Mgntud)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\user\Thesis\Classifying\CNNsplit\CNN_ECGNonInvasivefetal\augment.py:1504: ComplexWarning: Casting complex values to real discards the imaginary part
  return(np.float16(p_aug_raw))


62/62 [==============================] - 0s 2ms/step


KeyboardInterrupt: 

In [ ]:
rslts